# 先物データの可視化分析（静的グラフ版）

MatplotlibとSeabornを使用した美しい静的グラフでLME銅先物データを分析します。

In [ ]:
import sys
import os
import pandas as pd
import numpy as np
import pyodbc
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from matplotlib.gridspec import GridSpec
import seaborn as sns
import warnings

# プロジェクトのルートディレクトリをPythonパスに追加
project_root = os.path.dirname(os.path.dirname(os.path.abspath('__file__')))
sys.path.insert(0, project_root)

from config.database_config import get_connection_string

warnings.filterwarnings('ignore')

# カスタムスタイル設定
plt.style.use('seaborn-v0_8-darkgrid')
plt.rcParams['font.family'] = 'sans-serif'
plt.rcParams['font.sans-serif'] = ['Arial', 'DejaVu Sans']
plt.rcParams['font.size'] = 11
plt.rcParams['axes.labelsize'] = 12
plt.rcParams['axes.titlesize'] = 14
plt.rcParams['figure.titlesize'] = 16
plt.rcParams['figure.dpi'] = 100
plt.rcParams['savefig.dpi'] = 150
plt.rcParams['axes.grid'] = True
plt.rcParams['grid.alpha'] = 0.3

# カラーパレット
COLORS = {
    'primary': '#2E86AB',
    'secondary': '#A23B72',
    'accent': '#F18F01',
    'success': '#C73E1D',
    'dark': '#2D3436',
    'light': '#F7F7F7'
}

# グラデーションカラー
gradient_colors = ['#667eea', '#764ba2', '#f093fb', '#f5576c', '#4facfe', '#00f2fe']

print("📊 ライブラリのインポートが完了しました")

## データ取得

In [ ]:
def get_futures_data(conn, days=90):
    """先物データを取得"""
    query = f"""
    SELECT 
        p.TradeDate,
        m.MetalCode,
        m.ExchangeCode,
        t.TenorTypeName,
        p.SettlementPrice,
        p.Volume,
        p.OpenInterest,
        CASE 
            WHEN t.TenorTypeName LIKE 'Generic 1%' THEN 1
            WHEN t.TenorTypeName LIKE 'Generic 2%' THEN 2
            WHEN t.TenorTypeName LIKE 'Generic 3%' THEN 3
            WHEN t.TenorTypeName LIKE 'Generic 4%' THEN 4
            WHEN t.TenorTypeName LIKE 'Generic 5%' THEN 5
            WHEN t.TenorTypeName LIKE 'Generic 6%' THEN 6
            WHEN t.TenorTypeName LIKE 'Generic 7%' THEN 7
            WHEN t.TenorTypeName LIKE 'Generic 8%' THEN 8
            WHEN t.TenorTypeName LIKE 'Generic 9%' THEN 9
            WHEN t.TenorTypeName LIKE 'Generic 10%' THEN 10
            WHEN t.TenorTypeName LIKE 'Generic 11%' THEN 11
            WHEN t.TenorTypeName LIKE 'Generic 12%' THEN 12
            ELSE 0
        END as TenorNumber
    FROM T_CommodityPrice p
    INNER JOIN M_Metal m ON p.MetalID = m.MetalID
    INNER JOIN M_TenorType t ON p.TenorTypeID = t.TenorTypeID
    WHERE 
        t.TenorTypeName LIKE 'Generic%Future%'
        AND p.TradeDate >= DATEADD(day, -{days}, GETDATE())
        AND p.SettlementPrice IS NOT NULL
    ORDER BY p.TradeDate DESC, m.ExchangeCode, t.TenorTypeID
    """
    
    with warnings.catch_warnings():
        warnings.filterwarnings("ignore", message="pandas only supports SQLAlchemy")
        df = pd.read_sql(query, conn)
    
    df['TradeDate'] = pd.to_datetime(df['TradeDate'])
    return df

# データベース接続
conn = pyodbc.connect(get_connection_string())
print("✅ データベースに接続しました")

# データ取得
futures_df = get_futures_data(conn, days=90)
print(f"📈 {len(futures_df):,}件のデータを取得しました")
print(f"📅 期間: {futures_df['TradeDate'].min().strftime('%Y-%m-%d')} ～ {futures_df['TradeDate'].max().strftime('%Y-%m-%d')}")

# ExchangeCodeがNoneでないものだけを抽出
exchanges = [str(x) for x in futures_df['ExchangeCode'].unique() if x is not None]
if exchanges:
    print(f"🏢 取引所: {', '.join(exchanges)}")
else:
    print("🏢 取引所: データなし")

## 1. 先物カーブの時系列ヒートマップ

In [ ]:
# LMEデータのみ抽出
lme_data = futures_df[futures_df['ExchangeCode'] == 'LME'].copy()

# 最新30日分のデータでピボット
recent_dates = lme_data['TradeDate'].unique()[:30]
lme_recent = lme_data[lme_data['TradeDate'].isin(recent_dates)]

# ピボットテーブル作成
pivot_data = lme_recent.pivot_table(
    values='SettlementPrice',
    index='TradeDate',
    columns='TenorNumber',
    aggfunc='mean'
).sort_index(ascending=True)

# ヒートマップ
fig, ax = plt.subplots(figsize=(14, 8))

# カスタムカラーマップ
from matplotlib.colors import LinearSegmentedColormap
n = len(gradient_colors)
cmap = LinearSegmentedColormap.from_list('custom', gradient_colors, N=256)

im = ax.imshow(pivot_data.values, aspect='auto', cmap=cmap, interpolation='bilinear')

# 軸の設定
ax.set_xticks(range(len(pivot_data.columns)))
ax.set_xticklabels([f'M{i}' for i in pivot_data.columns])
ax.set_yticks(range(0, len(pivot_data.index), 5))
ax.set_yticklabels(pivot_data.index[::5].strftime('%m/%d'))

# カラーバー
cbar = plt.colorbar(im, ax=ax, pad=0.02)
cbar.set_label('価格 (USD/t)', rotation=270, labelpad=20)

# タイトルとラベル
ax.set_title('LME銅先物カーブの時系列変化', fontsize=16, weight='bold', pad=20)
ax.set_xlabel('限月', fontsize=12)
ax.set_ylabel('取引日', fontsize=12)

# グリッドラインを追加
ax.set_xticks(np.arange(len(pivot_data.columns)+1)-0.5, minor=True)
ax.set_yticks(np.arange(len(pivot_data.index)+1)-0.5, minor=True)
ax.grid(which='minor', color='white', linestyle='-', linewidth=1)

plt.tight_layout()
plt.show()

## 2. 取引所間価格比較

In [ ]:
# 1番限のデータのみ抽出
first_month = futures_df[futures_df['TenorNumber'] == 1].copy()
first_month = first_month.sort_values('TradeDate')

# グラフの作成
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(14, 10), 
                                gridspec_kw={'height_ratios': [3, 1]})

# 価格推移
for exchange, color in zip(['LME', 'SHFE', 'CMX'], 
                          [COLORS['primary'], COLORS['secondary'], COLORS['accent']]):
    exchange_data = first_month[first_month['ExchangeCode'] == exchange]
    if not exchange_data.empty:
        ax1.plot(exchange_data['TradeDate'], exchange_data['SettlementPrice'],
                label=exchange, color=color, linewidth=2.5, alpha=0.8)
        
        # 最新価格にマーカー
        latest = exchange_data.iloc[-1]
        ax1.scatter(latest['TradeDate'], latest['SettlementPrice'],
                   color=color, s=100, zorder=5)
        ax1.annotate(f'${latest["SettlementPrice"]:,.0f}',
                    xy=(latest['TradeDate'], latest['SettlementPrice']),
                    xytext=(10, 10), textcoords='offset points',
                    fontsize=10, color=color, weight='bold',
                    bbox=dict(boxstyle='round,pad=0.5', facecolor='white', 
                             edgecolor=color, alpha=0.8))

ax1.set_title('取引所別 銅先物価格推移（1番限）', fontsize=16, weight='bold')
ax1.set_ylabel('価格 (USD/t)', fontsize=12)
ax1.legend(loc='upper left', frameon=True, fancybox=True, shadow=True)
ax1.grid(True, alpha=0.3)

# フォーマット設定
ax1.xaxis.set_major_formatter(mdates.DateFormatter('%m/%d'))
ax1.xaxis.set_major_locator(mdates.DayLocator(interval=7))

# スプレッド（LME - SHFE）
lme_1m = first_month[first_month['ExchangeCode'] == 'LME'].set_index('TradeDate')['SettlementPrice']
shfe_1m = first_month[first_month['ExchangeCode'] == 'SHFE'].set_index('TradeDate')['SettlementPrice']

# 共通の日付でスプレッド計算
common_dates = lme_1m.index.intersection(shfe_1m.index)
spread = lme_1m[common_dates] - shfe_1m[common_dates]

ax2.fill_between(spread.index, spread, 0, 
                where=(spread >= 0), interpolate=True,
                color=COLORS['success'], alpha=0.5, label='LME > SHFE')
ax2.fill_between(spread.index, spread, 0,
                where=(spread < 0), interpolate=True,
                color=COLORS['primary'], alpha=0.5, label='SHFE > LME')
ax2.plot(spread.index, spread, color=COLORS['dark'], linewidth=2)
ax2.axhline(y=0, color='black', linestyle='--', alpha=0.5)

ax2.set_ylabel('スプレッド (USD/t)', fontsize=12)
ax2.set_xlabel('取引日', fontsize=12)
ax2.legend(loc='upper left', frameon=True)
ax2.grid(True, alpha=0.3)

# フォーマット設定
ax2.xaxis.set_major_formatter(mdates.DateFormatter('%m/%d'))
ax2.xaxis.set_major_locator(mdates.DayLocator(interval=7))

plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

## 3. 限月間スプレッド構造

In [ ]:
# 最新取引日のLMEデータ
latest_date = lme_data['TradeDate'].max()
latest_lme = lme_data[lme_data['TradeDate'] == latest_date].copy()

# 価格とスプレッドの計算
latest_lme = latest_lme.sort_values('TenorNumber')
prices = latest_lme.set_index('TenorNumber')['SettlementPrice']

# カスタムレイアウト
fig = plt.figure(figsize=(16, 10))
gs = GridSpec(3, 2, figure=fig, hspace=0.3, wspace=0.3)

# 1. 先物カーブ
ax1 = fig.add_subplot(gs[0, :])
x = list(prices.index)
y = list(prices.values)

# グラデーション塗りつぶし
ax1.fill_between(x, y, alpha=0.3, color=COLORS['primary'])
ax1.plot(x, y, color=COLORS['primary'], linewidth=3, marker='o', 
         markersize=8, markerfacecolor='white', markeredgewidth=2)

# 価格ラベル
for i, (tenor, price) in enumerate(prices.items()):
    if i % 2 == 0:  # 2つおきにラベル表示
        ax1.annotate(f'${price:,.0f}', (tenor, price), 
                    textcoords="offset points", xytext=(0,10), 
                    ha='center', fontsize=9, weight='bold')

ax1.set_title(f'LME銅先物カーブ ({latest_date.strftime("%Y-%m-%d")})',
             fontsize=14, weight='bold')
ax1.set_xlabel('限月', fontsize=12)
ax1.set_ylabel('価格 (USD/t)', fontsize=12)
ax1.set_xticks(x)
ax1.set_xticklabels([f'M{i}' for i in x])
ax1.grid(True, alpha=0.3)

# 2. カレンダースプレッド
ax2 = fig.add_subplot(gs[1, 0])
spreads = []
spread_labels = []
for i in range(1, len(prices)):
    spread = prices.iloc[i] - prices.iloc[i-1]
    spreads.append(spread)
    spread_labels.append(f'M{prices.index[i-1]}-M{prices.index[i]}')

colors_spread = [COLORS['success'] if s > 0 else COLORS['secondary'] for s in spreads]
bars = ax2.bar(range(len(spreads)), spreads, color=colors_spread, alpha=0.7)

# 値ラベル
for bar, val in zip(bars, spreads):
    height = bar.get_height()
    ax2.text(bar.get_x() + bar.get_width()/2., height + (5 if height > 0 else -15),
            f'${val:.0f}', ha='center', va='bottom' if height > 0 else 'top',
            fontsize=9, weight='bold')

ax2.set_title('隣接限月間スプレッド', fontsize=12, weight='bold')
ax2.set_xticks(range(len(spreads)))
ax2.set_xticklabels(spread_labels, rotation=45, ha='right')
ax2.set_ylabel('スプレッド (USD/t)', fontsize=11)
ax2.axhline(y=0, color='black', linestyle='-', linewidth=0.5)
ax2.grid(True, alpha=0.3, axis='y')

# 3. 期間構造の分類
ax3 = fig.add_subplot(gs[1, 1])
avg_spread = np.mean(spreads)
structure_type = "コンタンゴ" if avg_spread > 0 else "バックワーデーション"
color = COLORS['success'] if avg_spread > 0 else COLORS['secondary']

# 円グラフ風の表示
wedges, texts, autotexts = ax3.pie([abs(avg_spread), 100-abs(avg_spread)], 
                                    labels=[structure_type, ''],
                                    colors=[color, COLORS['light']],
                                    startangle=90,
                                    explode=(0.1, 0),
                                    autopct=lambda pct: f'{abs(avg_spread):.1f}' if pct > 50 else '')

ax3.set_title('期間構造', fontsize=12, weight='bold')

# 4. ボラティリティ
ax4 = fig.add_subplot(gs[2, :])
lme_vol = lme_data.copy()
lme_vol = lme_vol.sort_values(['TenorNumber', 'TradeDate'])
lme_vol['DailyReturn'] = lme_vol.groupby('TenorNumber')['SettlementPrice'].pct_change() * 100

# 限月別ボラティリティ
vol_by_tenor = lme_vol.groupby('TenorNumber')['DailyReturn'].std() * np.sqrt(252)

ax4.bar(vol_by_tenor.index, vol_by_tenor.values, 
       color=plt.cm.viridis(vol_by_tenor.values / vol_by_tenor.max()),
       alpha=0.8)

ax4.set_title('限月別ボラティリティ（年率換算）', fontsize=12, weight='bold')
ax4.set_xlabel('限月', fontsize=11)
ax4.set_ylabel('ボラティリティ (%)', fontsize=11)
ax4.set_xticks(vol_by_tenor.index)
ax4.set_xticklabels([f'M{i}' for i in vol_by_tenor.index])
ax4.grid(True, alpha=0.3, axis='y')

plt.suptitle('LME銅先物市場分析ダッシュボード', fontsize=18, weight='bold', y=0.98)
plt.tight_layout()
plt.show()

# 統計情報
print(f"\n📊 市場統計 ({latest_date.strftime('%Y-%m-%d')})")
print(f"期間構造: {structure_type}")
print(f"平均スプレッド: ${avg_spread:.2f}/月")
print(f"最大コンタンゴ: ${max(spreads):.2f} ({spread_labels[spreads.index(max(spreads))]})")
print(f"最大バックワード: ${min(spreads):.2f} ({spread_labels[spreads.index(min(spreads))]})")
print(f"平均ボラティリティ: {vol_by_tenor.mean():.1f}%")

## 4. 流動性分析

In [ ]:
# LMEの限月別出来高・建玉
lme_liquidity = lme_data.groupby('TenorNumber').agg({
    'Volume': ['mean', 'std'],
    'OpenInterest': ['mean', 'std']
}).round(0)

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 6))

# 出来高
tenors = lme_liquidity.index
volume_mean = lme_liquidity[('Volume', 'mean')]
volume_std = lme_liquidity[('Volume', 'std')]

bars1 = ax1.bar(tenors, volume_mean, yerr=volume_std, 
                color=gradient_colors[0], alpha=0.7,
                error_kw={'linewidth': 2, 'ecolor': 'gray', 'capsize': 5})

# グラデーション効果
for i, (bar, color) in enumerate(zip(bars1, gradient_colors[:len(bars1)])):
    bar.set_color(color)
    bar.set_alpha(0.8)

ax1.set_title('限月別平均出来高', fontsize=14, weight='bold')
ax1.set_xlabel('限月', fontsize=12)
ax1.set_ylabel('出来高（契約数）', fontsize=12)
ax1.set_xticks(tenors)
ax1.set_xticklabels([f'M{i}' for i in tenors])
ax1.grid(True, alpha=0.3, axis='y')

# 建玉
oi_mean = lme_liquidity[('OpenInterest', 'mean')]
oi_std = lme_liquidity[('OpenInterest', 'std')]

bars2 = ax2.bar(tenors, oi_mean, yerr=oi_std,
                color=gradient_colors[3], alpha=0.7,
                error_kw={'linewidth': 2, 'ecolor': 'gray', 'capsize': 5})

# グラデーション効果
for i, (bar, color) in enumerate(zip(bars2, gradient_colors[3:])):
    if i < len(gradient_colors) - 3:
        bar.set_color(color)
        bar.set_alpha(0.8)

ax2.set_title('限月別平均建玉', fontsize=14, weight='bold')
ax2.set_xlabel('限月', fontsize=12)
ax2.set_ylabel('建玉（契約数）', fontsize=12)
ax2.set_xticks(tenors)
ax2.set_xticklabels([f'M{i}' for i in tenors])
ax2.grid(True, alpha=0.3, axis='y')

plt.suptitle('LME銅先物 流動性分析', fontsize=16, weight='bold')
plt.tight_layout()
plt.show()

# 流動性集中度
total_volume = volume_mean.sum()
total_oi = oi_mean.sum()
front_3_volume = volume_mean[:3].sum() / total_volume * 100
front_3_oi = oi_mean[:3].sum() / total_oi * 100

print(f"\n📊 流動性分析")
print(f"フロント3限月の出来高集中度: {front_3_volume:.1f}%")
print(f"フロント3限月の建玉集中度: {front_3_oi:.1f}%")
print(f"最も流動性の高い限月: M{volume_mean.idxmax()}")

## データベース接続のクローズ

In [ ]:
conn.close()
print("✅ データベース接続をクローズしました")